In [ ]:
import numpy as np
import cv2
from cv2.ximgproc import guidedFilter
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
from skimage.io import imsave, imread
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error as mse
from skimage.color import rgb2hsv, hsv2rgb
import os
from dcp import *

In [ ]:
jlong_hyper = []
for i in range(1,11):
#     ref_fp = f'testing dataset/image{i}_ref.jpeg'
    for j in range(4):
        src_fp = f'../dataset/testing dataset/image{i}_haze{j}.jpeg'
        src=imread(src_fp)
        jlong_hyper.append(jlong(src, p=220))

In [ ]:
jlong_hyper_200 = []
for i in range(1,11):
#     ref_fp = f'testing dataset/image{i}_ref.jpeg'
    for j in range(4):
        src_fp = f'../dataset/testing dataset/image{i}_haze{j}.jpeg'
        src = imread(src_fp)
        jlong_hyper_200.append(jlong(src, p=200))

In [ ]:
metrics_long=[[], [], [], []]
metrics_long_200=[[], [], [], []]
for i in range(1,11):
    ref_fp = f'../dataset/testing dataset/image{i}_ref.jpeg'
    ref=imread(ref_fp)
    for j in range(4):
#         src_fp = f'testing dataset/image{i}_haze{j}.jpeg'
        res_jlong = jlong_hyper[j+(i-1)*4]
        res_jlong_200 = jlong_hyper_200[j+(i-1)*4]
#         print(res_jlong == res_jlong_200)
        metrics_long[j].append(calculate_metrics(ref, res_jlong))
        metrics_long_200[j].append(calculate_metrics(ref, res_jlong_200))

        
for i in range(1,11):
    ref_fp = f'testing dataset/image{i}_ref.jpeg'
    ref = imread(ref_fp)
    for j in range(4):
        src_fp = f'testing dataset/image{i}_haze{j}.jpeg'
        src = imread(src_fp)
        fig, ax = plt.subplots(1,4,figsize=(12,18))
        ax[0].set_title('Reference')
        ax[1].set_title('Simulated haze')
        ax[2].set_title(f'jLong 200 on {i} haze {j}')
        ax[3].set_title(f'jLong 220 on {i} haze {j}')
        ax[0].imshow(ref)
        ax[1].imshow(src)
        ax[2].imshow(jlong_hyper_200[j+(i-1)*4])
        ax[3].imshow(jlong_hyper[j+(i-1)*4])

In [ ]:
np.set_printoptions(suppress=True)

for (j_n, j) in enumerate(['uni   ', 'grad  ', 'c_uni ', 'c_grad']):
    mean = np.around(np.array(metrics_long[j_n]).mean(axis=0), decimals=3)
#     std = np.around(np.array(metrics[j_n]).std(axis=0), decimals=3)
#     med = np.around(np.median(np.array(metrics[i_n::6]), axis=0), decimals=3)
    mean_200 = np.around(np.array(metrics_long_200[j_n]).mean(axis=0), decimals=3)
    print("220 and 200 ", mean, mean_200)

In [ ]:
# обрезать - 600 на 600
# 10 картинок - 1 серая
# 4 - равномерных в некотором прямоугольнике
# 5 - градиентных 

In [ ]:
for i in ['idcp', 'long', 'zhu_dcp_gf','zhu_dcp_mean','zhu_dcp_paper', 'zhu_depth']:
    ref_fp = f'../dataset/real dehazed/9/dehazed9_{i}.jpeg'
    dst_fp = f'../dataset/real dehazed/9/cropped/dehazed9_{i}.jpeg'
    ref = imread(ref_fp)
    ref_cropped = ref[30:630, 10:610]
    imsave(dst_fp, ref_cropped.astype(np.uint8))
    
ref_fp = f'../dataset/real dehazed/9/real haze_9.jpeg'
dst_fp = f'../dataset/real dehazed/9/cropped/real haze_9.jpeg'
ref = imread(ref_fp)
ref_cropped = ref[30:630, 10:610]
imsave(dst_fp, ref_cropped.astype(np.uint8))

In [ ]:
ref_fp = f'dataset/real dehazed/10/real haze_10.jpeg'
dst_fp = f'dataset/real dehazed/11/real haze_10.jpeg'
ref = imread(ref_fp)
ref_cropped = ref[-1616:-51, 10:640]
imsave(dst_fp, ref_cropped.astype(np.uint8))

In [ ]:
#обрезка до размера 600 на 600
for i in range(1,9):
    ref_fp = f'../dataset/haze{i}_cut.jpeg'
    dst_fp = f'testing dataset/image{i}_ref.jpeg'
    ref = imread(ref_fp)
    ref_cropped = ref[10:610, 10:610]
    imsave(dst_fp, ref_cropped.astype(np.uint8))

for i in range(9,11):
    ref_fp = f'dataset/haze{i}_cut.jpeg'
    dst_fp = f'testing dataset/image{i}_ref.jpeg'
    ref = imread(ref_fp)
    ref_cropped = ref[-610:-10, -610:-10]
    imsave(dst_fp, ref_cropped.astype(np.uint8))

In [ ]:
#задымление
#0 - бесцветная равномерная, 1 - бесцветная градиентная, 2 - цветная равномерная, 3 - цветная градиентная
for i in range(1,11):
    ref_fp = f'testing dataset/image{i}_ref.jpeg'
    for j in range(4):
        dst_fp = f'testing dataset/image{i}_haze{j}.jpeg'
        t_min = np.random.rand() / 4 #from [0, 0.25]
        t_max = np.random.rand() / 4 + 0.75 #from [0.75, 1]
        if j==0: #achromatic  uniform
            hazing(ref_fp, dst_fp, [220, 220, 220], t_min, t_max, True)
        elif j==1: #achromatic  gradient
            hazing(ref_fp, dst_fp, [220, 220, 220], t_min, t_max, False)
        elif j==2: #uniform
            hazing(ref_fp, dst_fp, a_generator(), t_min, t_max, True)
        else: #gradient
            hazing(ref_fp, dst_fp, a_generator(), t_min, t_max, False)

In [ ]:
metrics=[[], [], [], []]
for i in range(1,11):
    ref_fp = f'testing dataset/image{i}_ref.jpeg'
    for j in range(4):
        src_fp = f'testing dataset/image{i}_haze{j}.jpeg'
        src = imread(src_fp)
        ref = imread(ref_fp)
        res_idcp = idcp(src)
        res_jlong = jlong(src)
        res_zhu_dcp_paper = zhu_dcp(src, gf_on=False, a_mean=False)
        res_zhu_dcp_mean = zhu_dcp(src, gf_on=False, a_mean=True)
        res_zhu_dcp_gf = zhu_dcp(src, gf_on=True, a_mean=True)
        res_zhu_depth = zhu_depth(src, gf_on=False) #a_mean=False
        
        metrics[j].append(calculate_metrics(ref, res_idcp))
        metrics[j].append(calculate_metrics(ref, res_jlong))
        metrics[j].append(calculate_metrics(ref, res_zhu_dcp_paper))
        metrics[j].append(calculate_metrics(ref, res_zhu_dcp_mean))
        metrics[j].append(calculate_metrics(ref, res_zhu_dcp_gf))
        metrics[j].append(calculate_metrics(ref, res_zhu_depth))
        print("IDCP res", calculate_metrics(ref, res_idcp))
        print("Long res", calculate_metrics(ref, res_jlong))
        print("Zhu+DCP res", calculate_metrics(ref, res_zhu_dcp_paper))
        print("Zhu+DCP+mean res", calculate_metrics(ref, res_zhu_dcp_mean))
        print("Zhu+DCP+gf res", calculate_metrics(ref, res_zhu_dcp_gf))
        print("Zhu+depth res", calculate_metrics(ref, res_zhu_depth))
        
        
        fig, ax = plt.subplots(1,8,figsize=(20,24))
        ax[0].set_title('Reference')
        ax[1].set_title('Simulated haze')
        ax[2].set_title('IDCP + guided filter')
        ax[3].set_title('Long')
        ax[4].set_title('Z. Zhu+DCP')
        ax[5].set_title('Z. Zhu+DCP+mean')
        ax[6].set_title('Z. Zhu+DCP+gf')
        ax[7].set_title('Z. Zhu+depth map')

        ax[0].imshow(ref)
        ax[1].imshow(src)
        ax[2].imshow(res_idcp)
        ax[3].imshow(res_jlong)
        ax[4].imshow(res_zhu_dcp_paper)
        ax[5].imshow(res_zhu_dcp_mean)
        ax[6].imshow(res_zhu_dcp_gf)
        ax[7].imshow(res_zhu_depth)

        plt.show()

In [ ]:
np.save('metrics.npy', np.array(metrics))

In [ ]:
import json

with open('metrics.txt', 'w') as filehandle:
    json.dump(metrics, filehandle)

In [ ]:
np.set_printoptions(suppress=True)

for (j_n, j) in enumerate(['uni   ', 'grad  ', 'c_uni ', 'c_grad']):
    for (i_n, i) in enumerate(['IDCP ', 'JLong', 'Z_DCP', 'Z_mn ', 'Z_gf ', 'Z_dpt']):
        mean = np.around(np.array(metrics[j_n][i_n::6]).mean(axis=0), decimals=3)
        std = np.around(np.array(metrics[j_n][i_n::6]).std(axis=0), decimals=3)
        med = np.around(np.median(np.array(metrics[j_n][i_n::6]), axis=0), decimals=3)
        print(f"Дымка {j}, метод {i} mean, med, std: SSIM", mean[0], med[0], std[0], 
              "MSE:", mean[1], med[1], std[1],
              "PSNR:", mean[2], med[2], std[2])              

In [ ]:
for i in range(11):
    src_fp = f'dataset/real haze/real haze_{i}.jpeg'
    dst_fp = f'dataset/real dehazed/{i}'
    src = imread(src_fp)
    res_zhu_depth = zhu_depth(src, gf_on=False) #a_mean=False

    imsave(os.path.join(dst_fp, f'dehazed{i}_zhu_depth.jpeg'), res_zhu_depth)

In [ ]:
for i in range(11):
    src_fp = f'dataset/real haze/real haze_{i}.jpeg'
    dst_fp = f'dataset/real dehazed'
    src = imread(src_fp)
    res_idcp = idcp(src)
    res_jlong = jlong(src)
    res_zhu_dcp_paper = zhu_dcp(src, gf_on=False, a_mean=False)
    res_zhu_dcp_mean = zhu_dcp(src, gf_on=False, a_mean=True)
    res_zhu_dcp_gf = zhu_dcp(src, gf_on=True, a_mean=True)
    res_zhu_depth = zhu_depth(src, gf_on=False) #a_mean=False

    imsave(os.path.join(dst_fp, f'dehazed{i}_idcp.jpeg'), res_idcp)
    imsave(os.path.join(dst_fp, f'dehazed{i}_long.jpeg'), res_jlong)
    imsave(os.path.join(dst_fp, f'dehazed{i}_zhu_dcp_paper.jpeg'), res_zhu_dcp_paper)
    imsave(os.path.join(dst_fp, f'dehazed{i}_zhu_dcp_mean.jpeg'), res_zhu_dcp_mean)
    imsave(os.path.join(dst_fp, f'dehazed{i}_zhu_dcp_gf.jpeg'), res_zhu_dcp_gf)
    imsave(os.path.join(dst_fp, f'dehazed{i}_zhu_depth.jpeg'), res_zhu_depth)

    
    
    
    fig, ax = plt.subplots(1,7,figsize=(15,21))
    ax[0].set_title('Real haze')
    ax[1].set_title('IDCP + guided filter')
    ax[2].set_title('Long')
    ax[3].set_title('Z. Zhu+DCP')
    ax[4].set_title('Z. Zhu+DCP+mean')
    ax[5].set_title('Z. Zhu+DCP+gf')
    ax[6].set_title('Z. Zhu+depth map')

    ax[0].imshow(src)
    ax[1].imshow(res_idcp)
    ax[2].imshow(res_jlong)
    ax[3].imshow(res_zhu_dcp_paper)
    ax[4].imshow(res_zhu_dcp_mean)
    ax[5].imshow(res_zhu_dcp_gf)
    ax[6].imshow(res_zhu_depth)

    plt.show()

In [ ]:
def idcp_plotting(I, dx=7, dy=7, k=0.95, t0=0.1, r=30, eps=0.01, dc_quantile=0.999):
# to do: check algorithm steps and parameters
# DCP by K. He: http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.672.3815&rep=rep1&type=pdf
    I_dark = get_dark_channel(I, dx, dy)
    q_I_dark = np.quantile(I_dark, dc_quantile)
    I_intens = I.sum(axis=2)
    q_intens = np.quantile(I_intens[I_dark >= q_I_dark], 0.9)
#     q_I = np.quantile(I[(I_dark >= q_I_dark)], 0.9, axis=0) #правильно ли работает с вектором
#     a_rgb = I[(I_dark >= q_I_dark) && (I >= q_I) ].mean(axis=0) #quantile not specified in the paper, надо бы некоторое усреднение добавить
    a_rgb = (I[(I_dark >= q_I_dark) & (I_intens>= q_intens) ]).mean(axis=0) 
    A = a_rgb * np.ones(I.shape)
#     print("IDCP Цвет дымки:", a_rgb)
    
    I_a = I / A
    V = get_dark_channel(I_a, dx, dy).astype(np.float32)
    t = 1 - k * V
    t_1 = guidedFilter(I, t, r, eps)
#     t = 1 - k * (1 - t)
    t = np.dstack((t_1, t_1, t_1))
    
    J = (I - A) / np.maximum(t, np.full(t.shape, t0)) + A
    J = np.clip(J, 0, 255).astype(np.uint8)
    
    
    
    
    return I, J, np.maximum(t_1, np.full(t_1.shape, t0)), A, (I_dark >= q_I_dark) & (I_intens>= q_intens)

In [ ]:
src_fp = f'dataset/real haze/real haze_1.jpeg'
src=imread(src_fp)

I, J, t, A, mask_dcp = idcp_plotting(src)

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(20,25))
axs[0].set_title('Исходное изображение')
axs[1].set_title('Карта пропускания')
axs[2].set_title('Атмосферное излучение')
axs[3].set_title('Результат')

axs[0].imshow(I.astype(np.uint8))
axs[1].imshow(t, cmap='cividis')
axs[2].imshow(A.astype(np.uint8))
axs[3].imshow(mask)

In [ ]:
def zhu_plotting(I, dx=7, dy=7, k=0.95, t0=0.01, r=30, eps=0.01, d_quantile=0.999, gf_on=True, a_mean=True):
# to do: check algorithm steps and parameters
# Atmospheric Light Estimation Based Remote Sensing Image Dehazing by
# Z. Zhu et. al.: https://www.mdpi.com/2072-4292/13/13/2432/htm

    d = zhu_depth_estim(I, dx, dy, r, eps, gf_on)
        
    q_d = np.quantile(d, d_quantile)
    I_intens = I.sum(axis=2)
    
    a_rgb_mean = (I[d >= q_d]).mean(axis=0) 
    I_max = I_intens[d >= q_d].max()
    a_rgb = I[(d >= q_d) & (I_intens == I_max)].mean(axis=0)
    A = a_rgb * np.ones(I.shape)
    A_mean = a_rgb_mean * np.ones(I.shape)

   
    return J, d >= q_d, (d >= q_d) & (I_intens == I_max), A, A_mean

In [ ]:
src_fp = f'testing dataset/image10_haze3.jpeg'
src=imread(src_fp)

I, mask_mean, mask_zhu, A_zhu, A_mean = zhu_plotting(src, gf_on=False)
I, J, t, A, mask_dcp = idcp_plotting(src)

In [ ]:
fig, axs = plt.subplots(1, 5, figsize=(20,25))
axs[0].set_title('(a)', x=0.5, y=-0.15, fontsize=20)
axs[1].set_title('(b)', x=0.5, y=-0.15, fontsize=20)
axs[2].set_title('(c)', x=0.5, y=-0.15, fontsize=20)
axs[3].set_title('(d)', x=0.5, y=-0.15, fontsize=20)
axs[4].set_title('(e)', x=0.5, y=-0.15, fontsize=20)

axs[0].set_xticks([])
axs[0].set_yticks([])
axs[1].set_xticks([])
axs[1].set_yticks([])
axs[2].set_xticks([])
axs[2].set_yticks([])
axs[3].set_xticks([])
axs[3].set_yticks([])
axs[4].set_xticks([])
axs[4].set_yticks([])

axs[0].imshow(I.astype(np.uint8))
axs[1].imshow(([230.2, 148.4, 140.5264443] * np.ones(src.shape)).astype(np.uint8))
axs[2].imshow(A.astype(np.uint8))
axs[3].imshow(A_zhu.astype(np.uint8))
axs[4].imshow(A_mean.astype(np.uint8))

In [ ]:
print(A, A_zhu, A_mean)

In [ ]:
def jlong_plotting(I, dx=7, dy=7, k=0.9, t0=0.1, sigma=2, dc_quantile=0.999, p=200):
# to do: check algorithm steps and parameters
# Single Remote Sensing Image Dehazing by
# J. Long et. al.: http://levir.buaa.edu.cn/publications/SingleRemoteSensingImageDehazing.pdf
    I_dark = get_dark_channel(I, dx, dy)
    q_I = np.quantile(I_dark, dc_quantile)
    a_rgb = (I[(I_dark >= q_I)]).mean(axis=0)
    A = a_rgb * np.ones(I.shape)
#     print("Long Цвет дымки:", a_rgb)
    
    I_a = I / A
    I_a = (I_a - I_a.min()) / (I_a.max() - I_a.min())
    
    V = get_dark_channel(I_a, 1, 1).astype(np.float32)
    V_filtered = gaussian_filter(V, sigma)
    assert V.min() >= 0 and V.max() <= 1
    
    t = 1 - V_filtered
    t = np.dstack((t, t, t))
    
    M = np.ones(t.shape) * p
    diff = M / np.abs(I_a - A)
    print(f"diff: min={diff.min()}, max={diff.max()}")
    t_corr = np.minimum(np.maximum(diff, 1) * t, 1)
#     t = np.minimum(np.maximum(M / np.abs(I - A), 1) * t, 1)
    
    J = k * A + (I - k * A) / np.maximum(t, np.full(t.shape, t0))
    J = np.clip(J, 0, 255).astype(np.uint8)
    
    J_corr = k * A + (I - k * A) / np.maximum(t_corr, np.full(t.shape, t0))
    J_corr = np.clip(J_corr, 0, 255).astype(np.uint8)
    return I, J, J_corr

In [ ]:
src_fp = f'dataset/real haze/real haze_9.jpeg'
src=imread(src_fp)

I, J, J_corr = jlong_plotting(src)

In [ ]:

fig, axs = plt.subplots(1, 3, figsize=(12,16))
axs[0].set_title('Исходное изображение')
axs[1].set_title('Без цветокоррекции')
axs[2].set_title('Результат ')

axs[0].imshow(I.astype(np.uint8))
axs[1].imshow(J)
axs[2].imshow(J_corr)


In [ ]:
src_fp = f'dataset/real haze/real haze_1.jpeg'
src=imread(src_fp)

d_GF = zhu_depth_estim(src, gf_on=True)
d = zhu_depth_estim(src, gf_on=False)

t_depth = np.exp(-1.2*d)
t_depth_GF = np.exp(-1.2*d_GF)

In [ ]:
J_depth = zhu_depth(src)

In [ ]:
I, J, t, A = idcp_plotting(src)

In [ ]:
t_depth = np.exp(-1.2*d)

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(20,25))
axs[0].set_title('(a)', x=0.5, y=-0.15, fontsize=20)
axs[1].set_title('(b)', x=0.5, y=-0.15, fontsize=20)
axs[2].set_title('(c)', x=0.5, y=-0.15, fontsize=20)
axs[3].set_title('(d)', x=0.5, y=-0.15, fontsize=20)
axs[0].set_xticks([])
axs[0].set_yticks([])
axs[1].set_xticks([])
axs[1].set_yticks([])
axs[2].set_xticks([])
axs[2].set_yticks([])
axs[3].set_xticks([])
axs[3].set_yticks([])

axs[0].imshow(I.astype(np.uint8))
axs[1].imshow(t_depth, cmap='cividis')
axs[2].imshow(t_depth_GF, cmap='cividis')
im = axs[3].imshow(t, cmap='cividis')
plt.colorbar(im, ax=axs.ravel().tolist(), shrink=0.15)


In [ ]:
for i in [1,3]:
    ref_fp = f'testing dataset/image{i}_ref.jpeg'
    for j in [0, 3]:
        src_fp = f'testing dataset/image{i}_haze{j}.jpeg'
        dst_fp = f'figures'
        src = imread(src_fp)
        ref = imread(ref_fp)
        res_idcp = idcp(src)
        res_jlong = jlong(src)
        res_zhu_dcp_paper = zhu_dcp(src, gf_on=False, a_mean=False)
        res_zhu_dcp_mean = zhu_dcp(src, gf_on=False, a_mean=True)
        res_zhu_dcp_gf = zhu_dcp(src, gf_on=True, a_mean=True)
        res_zhu_depth = zhu_depth(src, gf_on=False) #a_mean=False
        
        print("IDCP res", calculate_metrics(ref, res_idcp))
        print("Long res", calculate_metrics(ref, res_jlong))
        print("Zhu+DCP res", calculate_metrics(ref, res_zhu_dcp_paper))
        print("Zhu+DCP+mean res", calculate_metrics(ref, res_zhu_dcp_mean))
        print("Zhu+DCP+gf res", calculate_metrics(ref, res_zhu_dcp_gf))
        print("Zhu+depth res", calculate_metrics(ref, res_zhu_depth))
        
       
        imsave(os.path.join(dst_fp, f'dehazed{i}_haze{j}_idcp.jpeg'), res_idcp)
        imsave(os.path.join(dst_fp, f'dehazed{i}_haze{j}_long.jpeg'), res_jlong)
        imsave(os.path.join(dst_fp, f'dehazed{i}_haze{j}_zhu_dcp_paper.jpeg'), res_zhu_dcp_paper)
        imsave(os.path.join(dst_fp, f'dehazed{i}_haze{j}_zhu_dcp_mean.jpeg'), res_zhu_dcp_mean)
        imsave(os.path.join(dst_fp, f'dehazed{i}_haze{j}_zhu_dcp_gf.jpeg'), res_zhu_dcp_gf)
        imsave(os.path.join(dst_fp, f'dehazed{i}_haze{j}_zhu_depth.jpeg'), res_zhu_depth)

In [ ]:
for j in [2, 3, 5, 6, 9]:
    src_fp = f'dataset/real dehazed/{j}'
    src = imread(os.path.join(src_fp, f'real haze_{j}.jpeg'))
    res_idcp = imread(os.path.join(src_fp, f'dehazed{j}_idcp.jpeg'))
    res_jlong = imread(os.path.join(src_fp, f'dehazed{j}_long.jpeg'))
    res_zhu_dcp_paper = imread(os.path.join(src_fp, f'dehazed{j}_zhu_dcp_paper.jpeg'))
    res_zhu_dcp_mean = imread(os.path.join(src_fp, f'dehazed{j}_zhu_dcp_mean.jpeg'))
    res_zhu_dcp_gf = imread(os.path.join(src_fp, f'dehazed{j}_zhu_dcp_gf.jpeg'))
    res_zhu_depth = imread(os.path.join(src_fp, f'dehazed{j}_zhu_depth.jpeg'))
#     imsave(os.path.join(dst_fp, f'dehazed{i}_zhu_dcp_mean.jpeg'), res_zhu_dcp_mean)




    fig, ax = plt.subplots(1,3,figsize=(20,24))
    ax[0].set_title('Reference')
    ax[1].set_title('Simulated haze')
    ax[2].set_title('IDCP + guided filter')
    ax[3].set_title('Long')
    ax[4].set_title('Z. Zhu+DCP')
    ax[5].set_title('Z. Zhu+DCP+mean')
    ax[6].set_title('Z. Zhu+DCP+gf')
    ax[7].set_title('Z. Zhu+depth map')

    for k in range(8):
        ax[k].set_xticks([])
        ax[k].set_yticks([])

    ax[0].imshow(ref)
    ax[1].imshow(src)
    ax[2].imshow(res_idcp)
    ax[3].imshow(res_jlong)
    ax[4].imshow(res_zhu_dcp_paper)
    ax[5].imshow(res_zhu_dcp_mean)
    ax[6].imshow(res_zhu_dcp_gf)
    ax[7].imshow(res_zhu_depth)

    plt.show()

In [ ]:
for i in range(10):
    src_fp = f'testing dataset/image1_haze{i}.jpeg'
    ref_fp = 'testing dataset/image1_ref.jpeg'
    src = imread(src_fp)
    ref = imread(ref_fp)
    
    res_idcp = idcp(src)
    ssim_idcp = np.around(ssim(ref, res_idcp, channel_axis=2), decimals=4)
    mse_idcp = np.around(mse(ref, res_idcp), decimals=1)
    print(f'IDCP with guided filter SSIM={ssim_idcp} MSE={mse_idcp}')
    imsave(f'testing dataset/image1_haze{i}_idcp.jpeg', res_idcp.astype(np.uint8))

    res_jlong = jlong(src)
    ssim_jlong = np.around(ssim(ref, res_jlong, channel_axis=2), decimals=4)
    mse_jlong = np.around(mse(ref, res_jlong), decimals=1)
    print(f'J. Long et. al. SSIM={ssim_jlong} MSE={mse_jlong}')
    imsave(f'testing dataset/image1_haze{i}_Long.jpeg', res_jlong.astype(np.uint8))

    
    res_zhu_dcp = zhu_dcp(src)
    ssim_zhu_dcp = np.around(ssim(ref, res_zhu_dcp, channel_axis=2), decimals=4)
    mse_zhu_dcp = np.around(mse(ref, res_zhu_dcp), decimals=1)
    print(f'Z. Zhu et. al. with T(x) from IDCP SSIM={ssim_zhu_dcp} MSE={mse_zhu_dcp}')
    imsave(f'testing dataset/image1_haze{i}_zhu_dcp.jpeg', res_zhu_dcp.astype(np.uint8))

    
    res_zhu_depth = zhu_depth(src)
    ssim_zhu_depth = np.around(ssim(ref, res_zhu_depth, channel_axis=2), decimals=4)
    mse_zhu_depth = np.around(mse(ref, res_zhu_depth), decimals=1)
    print(f'Z. Zhu et. al. with T(x) from depth map SSIM={ssim_zhu_depth} MSE={mse_zhu_depth}')
    imsave(f'testing dataset/image1_haze{i}_zhu_depth.jpeg', res_zhu_depth.astype(np.uint8))

    
    fig, ax = plt.subplots(1,6,figsize=(15,18))
    ax[0].set_title('Reference')
    ax[1].set_title('Simulated haze')
    ax[2].set_title('IDCP with guided filter')
    ax[3].set_title('J. Long et. al.')
    ax[4].set_title('Z. Zhu et. al. with T(x) from IDCP')
    ax[5].set_title('Z. Zhu et. al. with T(x) from depth map')

    ax[0].imshow(ref)
    ax[1].imshow(src)
    ax[2].imshow(res_idcp)
    ax[3].imshow(res_jlong)
    ax[4].imshow(res_zhu_dcp)
    ax[5].imshow(res_zhu_depth)

    plt.show()

In [ ]:
h = np.random.rand()/6
s = np.random.rand()/2
v = np.random.rand()/2 + 0.5
a_hsv = [h,s,v]
a_rgb = hsv2rgb(a_hsv)
a_rgb = (a_rgb*255).astype(np.uint8)
# plt.imshow(a_rgb*np.ones(I.shape))